In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
root = "final.csv"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
class CsvDataset(Dataset):
    def __init__(self, path):
        super(CsvDataset).__init__()
        csv = pd.read_csv(path)
        csv_np = csv.to_numpy()
        for i in [1, 3, 4, 5, 6, 7, 8]:
            wordset = {word: idx for idx, word in enumerate(np.unique(csv_np[:,i]))}
            for row in range(len(csv_np)):
                csv_np[row][i] = wordset[csv_np[row][i]]
        self.ints = torch.from_numpy(np.array(csv_np[:,[1,3,4,5,6,7,8]], dtype="int"))
        self.floats = torch.from_numpy(np.array(list(map(lambda x: (x - np.array([0.61756694, 5.49324267, 4.76044624, 3.05992563])) / np.array([0.39076653, 1.28855836, 0.77676551, 0.51616518]), csv_np[:,[2,9,10,11]])), dtype="float")).float()
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
        
        
    def __getitem__(self,idx):
        return self.ints[idx],self.floats[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.ints)

In [5]:
data_length = len(pd.read_csv(root))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

train_dataset = CsvDataset(root)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [6]:
class CsvNet(nn.Module):
    def __init__(self):
        super(CsvNet, self).__init__()
        self.emb1 = torch.nn.Embedding(8, 20)
        self.emb2 = torch.nn.Embedding(11, 20)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 20)
        self.emb5 = torch.nn.Embedding(4, 20)
        self.emb6 = torch.nn.Embedding(4, 20)
        self.emb7 = torch.nn.Embedding(7, 20)
        self.act = nn.ReLU()
        self.fc = nn.Linear(4, 80)
        self.fc1 = nn.Linear(220, 8192)
        self.fc2 = nn.Linear(8192, 8192)
        self.fc3 = nn.Linear(8192, 4096)
        self.fc4 = nn.Linear(4096, 2048)
        self.fc5 = nn.Linear(2048, 1)
        self.dropout = nn.Dropout()
    
    def forward(self, x, y):
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        y = self.fc(y)
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        x = self.dropout(self.act(self.fc1(x)))
        x = self.dropout(self.act(self.fc2(x)))
        x = self.dropout(self.act(self.fc3(x)))
        x = self.dropout(self.act(self.fc4(x)))
        return self.fc5(x)

In [7]:
model = CsvNet().to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for X1, X2, Y in train_loader:
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()  # why we use zero_grad?
        prediction = model(X1, X2)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for X1_val, X2_val, Y_val in val_loader:
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(X1_val, X2_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 3541.6694, val_loss: 2386.6855
[Epoch:  2] Average loss: 2355.6426, val_loss: 1980.0905
[Epoch:  3] Average loss: 2082.0061, val_loss: 1884.0554
[Epoch:  4] Average loss: 2165.1892, val_loss: 1844.1340
[Epoch:  5] Average loss: 1705.9196, val_loss: 1546.4407
[Epoch:  6] Average loss: 1426.4427, val_loss: 1656.5193
[Epoch:  7] Average loss: 1637.2540, val_loss: 1528.8763
[Epoch:  8] Average loss: 1267.2279, val_loss: 1369.3713
[Epoch:  9] Average loss: 1089.5737, val_loss: 1391.4446
[Epoch: 10] Average loss: 1105.5872, val_loss: 1487.8833
[Epoch: 11] Average loss: 1269.2225, val_loss: 1618.3997
[Epoch: 12] Average loss: 1103.5343, val_loss: 1911.9358
[Epoch: 13] Average loss: 1071.3447, val_loss: 2387.8027
[Epoch: 14] Average loss: 1388.0154, val_loss: 2075.0862
[Epoch: 15] Average loss: 1402.5323, val_loss: 2324.1848
[Epoch: 16] Average loss: 1216.5284, val_loss: 1884.6111
[Epoch: 17] Average loss: 1806.2540, val_loss: 1679.0137
[Epoch: 18] Average loss: 1312.

[Epoch: 147] Average loss: 622.5332, val_loss: 974.6200
[Epoch: 148] Average loss: 816.7721, val_loss: 916.0792
[Epoch: 149] Average loss: 738.5664, val_loss: 969.5978
[Epoch: 150] Average loss: 904.2173, val_loss: 947.8285
[Epoch: 151] Average loss: 944.2325, val_loss: 888.8931
[Epoch: 152] Average loss: 833.4468, val_loss: 843.2939
[Epoch: 153] Average loss: 728.5841, val_loss: 978.0210
[Epoch: 154] Average loss: 733.9896, val_loss: 720.6635
[Epoch: 155] Average loss: 699.8944, val_loss: 853.6042
[Epoch: 156] Average loss: 731.2227, val_loss: 822.1558
[Epoch: 157] Average loss: 758.3587, val_loss: 923.1601
[Epoch: 158] Average loss: 785.2900, val_loss: 833.2980
[Epoch: 159] Average loss: 744.1187, val_loss: 916.8730
[Epoch: 160] Average loss: 721.9285, val_loss: 794.4692
[Epoch: 161] Average loss: 688.1365, val_loss: 854.9926
[Epoch: 162] Average loss: 1082.7083, val_loss: 1158.8125
[Epoch: 163] Average loss: 1846.7073, val_loss: 1230.9882
[Epoch: 164] Average loss: 1121.3134, val_lo

In [9]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for X1_test, X2_test, Y_test in test_loader:
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X1_test, X2_test)
        prices_mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - prices_mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 83.95%
